# Ошибки и исключения

# Оглавление


* [Синтаксические ошибки](#syntax_error)
* [Исключения](#exceptions)
* [Обработка исключений](#exceptions_handling)
* [Вызов исключений](#exceptions_call)
* [Пользовательские исключения](#custom_exceptions)
* [Определение последних действий](#last_actions)

До сих пор сообщения об ошибках было не более чем упоминанием. В этой главе мы подробнее поговорим о них и о том, как работать с ними. Существуют (по меньшей мере) два различимых вида ошибок: синтаксические ошибки и исключения.

# Синтаксические ошибки <a class="anchor" id="syntax_error"></a>

Синтаксические ошибки являются наиболее распространенным видом ошибок, с которыми встречаются люди изучающие Python

In [4]:
for i in range(3)
    print('Hello world')

SyntaxError: invalid syntax (1735228507.py, line 1)

Синтаксический анализатор повторяет строку, в которой была обнаружена ошибка, и отображает небольшую «стрелку», 
указывающую на самую раннюю точку в строке, где была обнаружена ошибка. 
Ошибка вызвана символом перед стрелкой: 
в этом примере ошибка обнаруживается в функции print(), поскольку перед ней отсутствует двоеточие (':'). 
Можно локализовать ошибку по сообщению с именем файла и номером строки.

# Исключения <a class="anchor" id="exceptions"></a>

Даже если оператор или выражение синтаксически верны, 
они могут вызвать ошибку при попытке их выполнить. 
Ошибки, обнаруженные во время выполнения, называются исключениями. 
Большинство исключений не обрабатываются программами и приводят к сообщениям об ошибках, как показано здесь:

In [5]:
10 * (1/0)

ZeroDivisionError: division by zero

In [6]:
4 + spam*3

NameError: name 'spam' is not defined

In [7]:
'2' + 2

TypeError: can only concatenate str (not "int") to str

Последняя строка сообщения об ошибке указывает, что произошло. Исключения бывают разных типов, и тип печатается как часть сообщения. Это верно для всех встроенных исключений, но не обязательно для пользовательских исключений (хотя это полезное соглашение). 

Остальная часть строки предоставляет подробную информацию в зависимости от типа исключения и его причины.

Первая часть сообщения об ошибке показывает контекст, в котором произошло исключение, в виде обратной трассировки стека. Как правило, он содержит список исходных строк трассировки стека

# Обработка исключений <a class="anchor" id="exceptions_handling"></a>

Можно писать скрипты, которые будут определенные типы исключений. Следующий пример запрашивает у пользователя ввод до тех пор, пока не будет введено действительное целое число, но позволяет пользователю прервать программу (используя Ctrl-C или чем-то еще, что поддерживает операционная система). Пользовательское прерывание вызывает исключение KeyboardInterrupt.

In [10]:
while True:
    try:
        x = int(input("Please enter a number: "))
        break
    except ValueError:
        print("Oops!  That was no valid number.  Try again...")

Please enter a number: й
Oops!  That was no valid number.  Try again...
Please enter a number: цу
Oops!  That was no valid number.  Try again...
Please enter a number: к
Oops!  That was no valid number.  Try again...
Please enter a number: 1


Оператор try работает следующим образом.

* Сначала выполняется тело оператора try (строки кода междуу операторами try и except).

* Если не возникает исключения, то тело оператора except пропускается и выполнение оператора try завершается.

* Если во время выполнения тела оператора try возникает исключение, то вся оставшаяся часть тела оператора пропускается. Если вызванное исключение соответствует исключению, названному после оператора except, то выполняется тело except, а затем выполняется оставшийся после блока try-except код.

* Если возникает исключение, которое не соответствует исключению, указанному в предложении except, то оно передается во внешние операторы try; если обработчик не найден, это необработанное исключение, и выполнение останавливается с сообщением, как показано выше.

У оператора try может быть более одного предложения except, чтобы указать обработчики для разных исключений. При получении исключения будет выполнено не более одного обработчика. Предложение except может называть несколько исключений в виде кортежа в скобках, например:

In [ ]:
except (RuntimeError, TypeError, NameError):
    pass

Класс в предложении except совместим с исключением, если это тот же класс или его базовый класс (но не наоборот - предложение except, в котором перечислен производный класс, несовместимо с базовым классом). Например, следующий код будет печатать B, C, D в таком порядке:

In [12]:
class B(Exception):
    pass

class C(B):
    pass

class D(C):
    pass

for cls in [B, C, D]:
    try:
        raise cls()
    except D:
        print("D")
    except C:
        print("C")
    except B:
        print("B")

B
C
D


Обратите внимание, что если бы предложения except шли бы в другом порядке (с первым исключением B), он бы напечатал B, B, B - срабатывает первое соответствующее предложение except.


In [14]:
class B(Exception):
    pass

class C(B):
    pass

class D(C):
    pass

for cls in [B, C, D]:
    try:
        raise cls()
    except B:
        print("B")
    except D:
        print("D")
    except C:
        print("C")

B
B
B


Последнее предложение except может не включать имя (s) исключения. Используйте этот прием с особой осторожностью, так как таким образом легко замаскировать настоящую ошибку программирования! Его также можно использовать для вывода сообщения об ошибке и повторного вызова исключения (что позволяет вызывающей стороне также обрабатывать исключение):

In [8]:
import sys

try:
    f = open('myfile.txt')
    s = f.readline()
    i = int(s.strip())
except OSError as err:
    print("OS error: {0}".format(err))
except ValueError:
    print("Could not convert data to an integer.")
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

OS error: [Errno 2] No such file or directory: 'myfile.txt'


Оператор try… except имеет необязательный оператор else, который, если он присутствует, должен следовать за всеми операторами except. Это полезно для кода, который должен выполняться, если предложение try не вызывает исключения. Например:

In [22]:
filename = 'workfile'

try:
    f = open(filename, 'r')
except OSError:
    print(f'cannot open {filename}')
else:
    print('else clause')

else clause


Когда возникает исключение, с ним может быть связано значение, также известное как аргумент исключения.

Предложение except может указывать переменную после имени исключения. Переменная привязана к экземпляру исключения с аргументами, хранящимися в instance.args. Для удобства экземпляр исключения определяет метод __str __(), поэтому аргументы могут быть напечатаны напрямую, без ссылки на .args. Можно также сначала создать экземпляр исключения перед его вызовом и добавить к нему любые атрибуты по желанию.

In [23]:
try:
    raise Exception('spam', 'eggs')
except Exception as inst:
    print(type(inst))    # the exception instance
    print(inst.args)     # arguments stored in .args
    print(inst)          # __str__ allows args to be printed directly,
                         # but may be overridden in exception subclasses
    x, y = inst.args     # unpack args
    print('x =', x)
    print('y =', y)

<class 'Exception'>
('spam', 'eggs')
('spam', 'eggs')
x = spam
y = eggs


Если у исключения есть аргументы, они печатаются как последняя часть сообщения для необработанных исключений.

Обработчики исключений обрабатывают исключения не только в том случае, если они возникают сразу в предложении try, но также и в тех случаях, когда они возникают внутри функций, которые вызываются (даже косвенно) в предложении try. Например:

In [23]:
def this_fails():
    x = 1/0

try:
    this_fails()
except ZeroDivisionError as err:
    print('Handling run-time error:', err)

Handling run-time error: division by zero


# Вызов исключений <a class="anchor" id="exceptions_call"></a>

Оператор raise позволяет вызвать необходимое исключение. Например:

In [26]:
raise NameError('HiThere')

NameError: HiThere

Единственный аргумент для **raise** указывает на исключение, которое надо вызвать. Это должен быть либо экземпляр исключения, либо класс исключения (класс, производный от Exception). Если передан класс исключения, он будет неявно создан путем вызова его конструктора без аргументов:

In [27]:
raise ValueError  # shorthand for 'raise ValueError()'

ValueError: 

Если вам нужно определить, возникло ли исключение, но вы не собираетесь его обрабатывать, более простая форма оператора **raise** позволяет повторно вызвать исключение:

In [32]:
try:
    raise NameError('HiThere')
except NameError:
    print('An exception flew by!')
    raise

An exception flew by!


NameError: HiThere

# Пользовательские исключения <a class="anchor" id="custom_exceptions"></a>

В программах можно прописывать свои собственные исключения, создавая новый класс исключения (Exception). Исключения обычно должны быть производными от класса Exception, прямо или косвенно.

Могут быть определены классы исключений, которые делают все, что может делать любой другой класс, но обычно остаются простыми, часто предлагая только ряд атрибутов, которые позволяют обработчикам исключения извлекать информацию об ошибке. При создании модуля, который может вызывать несколько различных ошибок, обычной практикой является создание базового класса для исключений, определенных этим модулем, и подкласса, который создает определенные классы исключений для различных условий ошибки:

In [35]:
class Error(Exception):
    """Base class for exceptions in this module."""
    pass

class InputError(Error):
    """Exception raised for errors in the input.

    Attributes:
        expression -- input expression in which the error occurred
        message -- explanation of the error
    """

    def __init__(self, expression, message):
        self.expression = expression
        self.message = message

class TransitionError(Error):
    """Raised when an operation attempts a state transition that's not
    allowed.

    Attributes:
        previous -- state at beginning of transition
        next -- attempted new state
        message -- explanation of why the specific transition is not allowed
    """

    def __init__(self, previous, next, message):
        self.previous = previous
        self.next = next
        self.message = message

Большинство исключений определяются с именами, оканчивающимися на «Error», аналогично именованию стандартных исключений.

Многие стандартные модули определяют свои собственные исключения, чтобы сообщать об ошибках, которые могут возникнуть в функциях, которые они определяют.

# Определение последних действий <a class="anchor" id="last_actions"></a>

У оператора try есть еще одно необязательный оператор, который предназначен для определения последних действий, которые должны выполняться при любых обстоятельствах. Например: 

In [37]:
try:
    raise KeyboardInterrupt
finally:
    print('Goodbye, world!')

Goodbye, world!


KeyboardInterrupt: 

Предложение finally всегда выполняется перед выходом из оператора try, независимо от того, произошло ли исключение. Когда исключение произошло в предложении try и не было обработано предложением except (или оно произошло в предложении except или else), оно повторно возникает после выполнения предложения finally.  

* Если во время выполнения блока try возникает исключение, исключение может быть обработано блоком except. Если исключение не обрабатывается блоком except, исключение создается повторно после выполнения предложения finally.

* Исключение может возникнуть во время выполнения блоков except или else. Опять же, исключение возникает повторно после выполнения блока finally.

* Если блок finally выполняет оператор break, continue или return, исключения не вызываются повторно.

* Если оператор try достигает оператора break, continue или return, блок finally будет выполнен непосредственно перед выполнением оператора break, continue или return.

* Если блок finally включает оператор return, возвращаемое значение будет значением из оператора return блока finally, а не значением из оператора return блок try.

Еще один пример:

In [38]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("division by zero!")
    else:
        print("result is", result)
    finally:
        print("executing finally clause")

divide(2, 1)


divide(2, 0)


divide("2", "1")

result is 2.0
executing finally clause
division by zero!
executing finally clause
executing finally clause


TypeError: unsupported operand type(s) for /: 'str' and 'str'

Как видите, предложение finally выполняется в любом случае. Ошибка TypeError, возникающая при разделении двух строк, не обрабатывается предложением except и поэтому повторно возникает после выполнения предложения finally.

В реальных приложениях предложение finally полезно для освобождения внешних ресурсов (таких как файлы или сетевые подключения), независимо от того, было ли использование ресурса успешным.